In [ ]:
!pip install textacy
!pip install rake-nltk
 
import networkx as nx
import re 
import itertools
import operator
import copy
import heapq
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
import string
import re
import matplotlib.pyplot as plt
from networkx import k_core
from networkx import core_number
import spacy
import sys
import numpy as np
import textacy
from nltk.tokenize import word_tokenize,wordpunct_tokenize
import networkx.drawing
#import textacy.keyterms
from textacy import preprocessing
from networkx import common_neighbors
import pandas as pd
from collections import OrderedDict
nltk.download('stopwords')
 
nlp = spacy.load('en')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install emoji
nb_keys_terms_needed=50
stop_words = set(stopwords.words('english'))
from google.colab import drive
#drive.mount('/content/drive')
#tweetdf = pd.read_csv('/content/drive/My Drive/opntwweets.csv')
tweetdf = pd.read_csv('/content/opntwweets.csv')
print(len(tweetdf))

txt_sentences = tweetdf[0:9].tweet.values
print(txt_sentences)

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
spacy_nlp = spacy.load('en_core_web_sm')
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)
punct = set(string.punctuation)
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
ignore_chars = set(['\n',' ','...','"',''])
dig_pattern = re.compile('[0-9]')
cleaned_text=[]
for i,phrase in enumerate(txt_sentences):
  tokens=[]
  phrase = re.sub(r"http\S+", "", phrase)
  phrase = dig_pattern.sub(r'',phrase)
  ed_phrase = emoji_pattern.sub(r'', phrase)
  doc = spacy_nlp(ed_phrase)
  tokens = [token.text for token in doc]
  tokens = [token for token in tokens if token not in stop_words]
  tokens = [token for token in tokens if token not in punct]
  tokens = [token for token in tokens if token not in ignore_chars]
  cleaned_text.append(tokens)

print(cleaned_text)

In [ ]:
phrasesdf = pd.read_csv('/content/drive/My Drive/texts - Sheet1.csv')
phrasesdf.columns = ['text']

tweets = phrasesdf.text.values
text = ''
for tweet in tweets:
  text+=tweet


In [ ]:
def terms_to_graph_sents(clean_txt_sents, w,stopping_end_of_line=0):
    from_to = {}
    if(not stopping_end_of_line):
        extended_clean_txt_sents=[]
        for sublist in clean_txt_sents:
            extended_clean_txt_sents.extend(sublist)
        clean_txt_sents=[extended_clean_txt_sents]
    for k,sents in enumerate(clean_txt_sents):
        len_sents=len(sents)

        # create initial complete graph (first w terms)
        terms_temp = sents[0:min(w,len_sents)]
        indexes = list(itertools.combinations(range(min(w,len_sents)), r=2))
        new_edges = []
        for my_tuple in indexes:
            new_edges.append(tuple([terms_temp[i] for i in my_tuple]))

        for new_edge in new_edges:
            if new_edge in from_to:
                from_to[new_edge] += 1
            else:
                from_to[new_edge] = 1

        if(w<=len_sents):
            # then iterate over the remaining terms
            for i in range(w, len_sents):
                considered_term = sents[i] # term to consider
                terms_temp = sents[(i-w+1):(i+1)] # all terms within sliding window

                # edges to try
                candidate_edges = []
                for p in range(w-1):
                    candidate_edges.append((terms_temp[p],considered_term))

                for try_edge in candidate_edges:

                    if try_edge[1] != try_edge[0]:
                    # if not self-edge

                        # if edge has already been seen, update its weight
                        if try_edge in from_to:
                            from_to[try_edge] += 1

                        # if edge has never been seen, create it and assign it a unit weight     
                        else:
                            from_to[try_edge] = 1
    
    return(from_to)

tuples_words_sents_unweighted=list(terms_to_graph_sents(cleaned_text, w=5,stopping_end_of_line=1).keys())
tuples_words_sents_weighted=terms_to_graph_sents(cleaned_text, w=5,stopping_end_of_line=0)

print("tuples unweighted \n")
tuples_words_sents_unweighted[:10]

In [ ]:
nb_iter=0
print("tuples weighted \n")
for key in tuples_words_sents_weighted.keys():
    if(nb_iter <10):
        nb_iter+=1
        print(key,tuples_words_sents_weighted[key])

In [ ]:

## UNWEIGHTED AND WEIGHTED GRAPH 
def unweighted_graph(tuples_words_sents_unweighted):
    G = nx.Graph()
    G.add_edges_from(tuples_words_sents_unweighted)
    G.remove_edges_from(nx.selfloop_edges(G))
    return(G)

G_unweighted=unweighted_graph(tuples_words_sents_unweighted)

def weighted_graph(tuples_words_sents_weighted):
    G = nx.Graph()
    for keys,values in tuples_words_sents_weighted.items():
        G.add_edge(keys[0],keys[1],weight=values)
        G.remove_edges_from(nx.selfloop_edges(G))
    return(G)

G_weighted=weighted_graph(tuples_words_sents_weighted)

In [ ]:
## Generate and order a list of best keywords, the number is specified by "nb_keys_terms_needed"
G_Unweighted_core_number=core_number(G_unweighted)
G_Weighted_core_number=core_number(G_weighted)

def order_dict_best_keywords(G_core_number,nb_keys_terms_needed=10):
    k_core_keyTerms=sorted(G_core_number, key=G_core_number.get, reverse=True)
    Kcore_values=[G_core_number[x] for x in k_core_keyTerms[:nb_keys_terms_needed]]
    return(k_core_keyTerms,Kcore_values)

UW_k_core_keyTerms,UW_Kcore_values=order_dict_best_keywords(G_Unweighted_core_number,nb_keys_terms_needed)
W_k_core_keyTerms,W_Kcore_values=order_dict_best_keywords(G_Weighted_core_number,nb_keys_terms_needed)
print(UW_k_core_keyTerms)

In [ ]:
## WEIGHTED 
nx.draw(G_weighted, with_labels=True,font_size=0.02,width=1,node_size=10,label='Graph Unweighted GOT')
plt.show()

In [ ]:

## UNWEIGHTED
nx.draw(G_unweighted, with_labels=True,font_size=0.02,width=1,node_size=10,label='Graph Unweighted GOT')
plt.show()

In [ ]:
subgraph_k_core=k_core(G_weighted, k=10 ,core_number=None)
nx.draw(subgraph_k_core, with_labels=True,node_color='y',edge_color='g',font_size=10,width=1,node_size=500,label='Graph Unweighted GOT')
plt.show()
plt.savefig("subrgraph_kcore.png")

In [ ]:
!pip install ir-evaluation-py
import csv
from networkx import closeness_centrality,degree_centrality
from collections import Counter
G=G_unweighted.copy()
def compute_sup_e(H):
    dict_res={}
    edges_sorted={}
    edges=H.edges
    for e in edges:
        edges_sorted[e]=len(list(common_neighbors(H, e[0], e[1])))
    edges_sorted=sorted(edges_sorted.items(), key=lambda x: x[1])
    return(OrderedDict(edges_sorted))

def dict_to_sorted_dict(dictionnary):
    return(OrderedDict(sorted(dictionnary.items(), key=lambda x: x[1])))
 
def k_truss_True_Dec_Massive_Net(G,output_for_density=True):
    edges_sorted=compute_sup_e(G)
    print(edges_sorted)
    keys, values = [], []

    for key, value in edges_sorted.items():
      keys.append(key)
      values.append(value)       
    print(len(values))
    ranks, counts = np.unique(values, return_counts=True)
    print(dict(zip(ranks,counts)))
    with open("frequencies.csv", "w") as outfile:
      csvwriter = csv.writer(outfile)
      csvwriter.writerow(keys)
      csvwriter.writerow(values)
    k_truss_nodes={}
    for node in G.nodes:
        k_truss_nodes[node]=0
    not_all_edges_removed=True
    edges_sorted=compute_sup_e(G)
    k=2
    liste_remove=[]
    output_density=[[k,len(G.nodes()),len(G.edges)]]
    while (not_all_edges_removed):
        liste_remove_k=[]
        while(list(edges_sorted.values())[0]<=k-2):
            e=list(edges_sorted.keys())[0]
            node1=list(edges_sorted.keys())[0][0]
            node2=list(edges_sorted.keys())[0][1]
            nb_u=G.neighbors(node1)
            nb_v=G.neighbors(node2)
            nbU=list(nb_v)
            u=node2
            v=node1
            if (len(list(nb_u))<len(list(nb_v))) :
                nbU=list(nb_u)
                u=node1
                v=node2
            for w in nbU:
                if(G.has_edge(w,v)):
                    sup_keys=edges_sorted.keys()
                    if((v,w) in sup_keys):
                        edges_sorted[(v,w)]-=1
                    else:
                        edges_sorted[(w,v)]-=1
                    if((u,w) in sup_keys):
                        edges_sorted[(u,w)]-=1
                    else:
                        edges_sorted[(w,u)]-=1
                    edges_sorted=dict_to_sorted_dict(edges_sorted)
            del(edges_sorted[e])
            liste_remove_k.append(e)
            G.remove_edge(e[0],e[1])
            if(len(G.edges)==0):
                break
        liste_remove.append(liste_remove_k)
        
        if(len(list(edges_sorted.keys()))==0):
            not_all_edges_removed=False
        else:
            k+=1            
            G_bis=max((G.subgraph(c) for c in nx.connected_components(G)),key=len)
            for node_not_removed in G_bis.nodes:
                k_truss_nodes[node_not_removed]+=1
            if(output_for_density):
                output_density.append([k,len(list(G_bis.nodes)),len(G_bis.edges)]) 

            nx.draw(G_bis, with_labels=True,font_color='k',node_color='g',edge_color='y',font_size=max(min(20,500/len(G_bis.nodes)),9),width=1,node_size=0,label='Graph Unweighted GOT')
            #plt.figure(figsize=(15, 15))
            plt.savefig("Approach_3°{}.png".format(k-2))
            plt.show()
            
    return(k_truss_nodes,k,G_bis,output_density)
    
k_truss_nodes,k,G_bis,output_density=k_truss_True_Dec_Massive_Net(G,output_for_density=True)



In [ ]:
uw_some_word_list = closeness_centrality(G_unweighted)
uw_some_words = []
for key in uw_some_word_list.keys(): 
        uw_some_words.append(key)

un_deg_list = degree_centrality(G_unweighted)
un_deg_words = []
for key in un_deg_list.keys(): 
        un_deg_words.append(key)


sort_words = sorted(uw_some_word_list.items(), key=lambda x: x[1], reverse=True)


import pandas as pd 
  
# create DataFrame using data 
sw_df = pd.DataFrame(sort_words, columns =['Word','Score']) 
sw_df.to_csv('sort_words.csv')
print(sw_df) 

for_plot = sw_df.iloc[0:100]
for_plot.plot.density(color='Black')

for_plot.plot.hist(color='Black')


from networkx import pagerank
pr_list = nx.pagerank(G_unweighted)
pr_sort_words = sorted(pr_list.items(), key=lambda x: x[1], reverse=True)


import pandas as pd 
  
# create DataFrame using data 
pr_sw_df = pd.DataFrame(pr_sort_words, columns =['Word','Score']) 
pr_sw_df.to_csv('pr_sort_words.csv')
print(pr_sw_df)  